# Graph State Example

This example shows how to enact and measure a graph state on a set of connected qubits that form a graph with the qubits as nodes and pairs of coupled qubits as edges. 

## setup

In [ ]:
import os
from typing import List

import networkx as nx
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pyquil.api import get_qc, QuantumComputer

from forest_qcvv.compilation import basic_compile
from forest_qcvv.graph_state import create_graph_state, measure_graph_state

## graph state measurement and plotting functions

Here, we make programs on a graph with qubits as nodes on a quantum device. 

In [ ]:
def run_graph_state(qc: QuantumComputer,
                    nodes: List[int],
                    graph: nx.classes.graph.Graph,
                    n_shots: int = 1000):
    assert all([node in qc.qubits() for node in nodes]), "One or more nodes provided does not fall in the graph"
    
    results = []
    
    for node in nodes:
        print(f"Running graph state on QC{node}")
        program = create_graph_state(graph)
        measure_prog, c_addrs = measure_graph_state(graph, focal_node=node)
        program += measure_prog
        program = basic_compile(program)
        program.wrap_in_numshots_loop(n_shots)
        executable = qc.compile(program)

        qc.qam.load(executable)
        for theta in np.linspace(-np.pi, np.pi, 21):
            qc.qam.write_memory(region_name='theta', value=theta)
            bitstrings = qc.qam.run().wait().read_from_memory_region(region_name='ro')
            parities = np.sum(bitstrings, axis=1) % 2
            avg_parity = np.mean(parities)
            results.append({
                'focal_node': node,
                'theta': theta,
                'n_bitstrings': len(bitstrings),
                'avg_parity': float(avg_parity),
            })

    pd.DataFrame(results).to_json('graph-state.json')

In [ ]:
def plot_graph_state():
    from matplotlib import pyplot as plt
    df = pd.read_json('graph-state.json')
    for focal_node in df['focal_node'].unique():
        df2 = df[df['focal_node'] == focal_node].sort_values('theta')
        plt.plot(df2['theta'], df2['avg_parity'], 'o-', label=f'{focal_node}')

    plt.legend(loc='best')
    plt.xlabel('theta')
    plt.ylabel('parity')
    plt.tight_layout()
    plt.show()

## creating a graph on a Rigetti lattice and running graph state measurements on it

We start with some connected qubits. For a simple example, we'll take three qubits arranged in a line, forming a three-vertex path, which we can represent in avant-garde ASCII art as follows:

    1 - 2 - 3
    
We define a graph with our qubits as nodes and pairs of coupled qubits as edges, so we have `[1, 2, 3]` as our nodes and `[(1, 2), (2, 3)]` as our edges. `networkx` has handy utilities for defining graphs from sets of edges and visualizing them so that we don't have to rely on my lackluster ASCII art. 

In [ ]:
# define the graph by its nodes and edges
nodes = [1, 2, 3]
graph = nx.from_edgelist([(1, 2), (2, 3)])

# make a figure representing the graph
nx.draw_networkx(graph)

# hide axis labels and figure outline
_ = plt.axis("off")

Now we can run the graph state measurement on each of the qubits in the simple, linear lattice we've specified. 

In [ ]:
nodes = [1, 2, 3]
graph = nx.from_edgelist([(1, 2), (2, 3)])

qc = get_qc('9q-square-qvm')

if not os.path.exists('graph-state.json'):
    run_graph_state(qc, nodes, graph)
    
plot_graph_state()